# Save study evaulation preset configurations

*2023-04-04*

Save study evaulation configuration for an given assessment if a format that can be used to load a configuration into HAWC. This notebook makes it easy to sync the current state of the scientific methodology for study evaluation and save as a default configuration in HAWC.

In [ ]:
import json
from pathlib import Path
from getpass import getpass

from hawc_client import HawcClient

## Configuration

What HAWC website do we use? What asessment? Where do we write the output?

In [ ]:
client = HawcClient(input(prompt="Domain:"))
client.set_authentication_token(getpass(prompt="Token:"))

In [ ]:
assessment_id = 12345
export_location = Path('example.json')

## Execution

Content below shouldn't need to be edited; it should properly build the format based on the configuration above.

In [ ]:
url = f"{client.session.root_url}/rob/api/assessment/{assessment_id}/settings/"
settings = client.session.get(url).json()
domains = sorted(settings['domains'], key=lambda d: d['sort_order'])

In [ ]:
url = f"{client.session.root_url}/rob/api/metric/?assessment_id={assessment_id}"
data = client.session.get(url).json()
metrics = sorted(data, key=lambda d: d['sort_order'])

In [ ]:
# build serialization format
data = {
    "help_text": settings["rob_settings"]["help_text"],
    "domains": [
        {            
            "name": domain["name"],
            "description": domain["description"],
            "is_overall_confidence": domain["is_overall_confidence"],
            "metrics": [
                {                    
                    # "key": metric["key"],
                    "key": -1,
                    "name": metric["name"],
                    "short_name": metric["short_name"],
                    "description": metric["description"],
                    "responses": metric["responses"],
                    "required_animal": metric["required_animal"],
                    "required_epi": metric["required_epi"],
                    "required_invitro": metric["required_invitro"],
                    "hide_description": metric["hide_description"],
                    "use_short_name": metric["use_short_name"],
                } for metric in metrics if metric['domain'] == domain['id']
            ]            
        } for domain in domains
    ]
}

In [ ]:
# write to disk
export_location.expanduser().resolve().write_text(json.dumps(data, indent=2))